## 📦 필요한 라이브러리 설치

In [ ]:
!pip install --upgrade google-api-python-client pandas google-auth-oauthlib google-auth-httplib2

## ✅ 분석 코드

In [ ]:
# -*- coding: utf-8 -*-
"""
YouTube 댓글 수집 스크립트 (Colab용)

지정한 검색어(제품명 등)로 YouTube 영상을 검색하고,
검색된 인기 영상들로부터 댓글 데이터를 수집.
"""

# 2. 라이브러리 임포트
import os
import pandas as pd
from google.colab import userdata # Colab의 Secrets 기능 사용
import googleapiclient.discovery
import googleapiclient.errors

# 3. YouTube API 키 설정
# !! 중요 !!
# Colab 좌측 메뉴의 'Secrets'(열쇠 모양 아이콘)에
# 'YOUTUBE_API_KEY' 라는 이름으로 자신의 YouTube Data API v3 키를 저장해서 진행
# 즉 이거 따로 돌릴 때 자기껄로 발급 받아서 넣어야혀요
try:
    API_KEY = userdata.get('YOUTUBE_API_KEY')
    if not API_KEY:
        raise ValueError("API Key not found in Colab Secrets.")
    print("YouTube API 키를 성공적으로 로드했습니다.")
except Exception as e:
    print(f"오류: Colab Secrets에서 'YOUTUBE_API_KEY'를 찾을 수 없습니다. {e}")
    print("Colab 좌측 메뉴 'Secrets'에 API 키를 추가해주세요.")
    # 스크립트 실행 중단 또는 기본값 설정 등의 처리가 필요할 수 있음
    API_KEY = None # 예외 발생 시 API_KEY를 None으로 설정

# 4. YouTube API 클라이언트 빌드 함수
def get_youtube_client(api_key):
    """YouTube API 서비스 클라이언트를 생성합니다."""
    if not api_key:
        print("오류: API 키가 유효하지 않아 YouTube 클라이언트를 생성할 수 없습니다.")
        return None
    api_service_name = "youtube"
    api_version = "v3"
    try:
        youtube = googleapiclient.discovery.build(
            api_service_name, api_version, developerKey=api_key)
        return youtube
    except Exception as e:
        print(f"YouTube 클라이언트 생성 중 오류 발생: {e}")
        return None

# 5. 동영상 검색 함수
def search_videos(youtube, query, max_results=5):
    """
    주어진 검색어(query)를 기반으로 YouTube 동영상을 검색합니다.

    Parameters:
        youtube (Resource): YouTube API 클라이언트
        query (str): 검색할 키워드 (예: "갤럭시 S24 후기")
        max_results (int): 검색할 최대 영상 수 (기본값: 5)

    Returns:
        List[Dict]: 검색된 영상들의 video_id와 video_title을 담은 리스트
    """
    if not youtube:
        print("YouTube 클라이언트가 유효하지 않습니다.")
        return []

    try:
        response = youtube.search().list(
            part="snippet",
            q=query,
            type="video",
            order="relevance",  # 관련성 기준 정렬
            maxResults=max_results
        ).execute()

        videos = []
        for item in response.get('items', []):
            video_id = item.get('id', {}).get('videoId')
            video_title = item.get('snippet', {}).get('title')
            if video_id and video_title:
                videos.append({
                    "video_id": video_id,
                    "video_title": video_title
                })

        print(f"[{query}] 검색 결과: {len(videos)}개 영상 발견")
        return videos

    except googleapiclient.errors.HttpError as e:
        if e.resp.status == 403:
            print(f"[{query}] API 권한 문제 또는 할당량 초과: {e}")
        else:
            print(f"[{query}] HTTP 오류 발생: {e}")
        return []

    except Exception as e:
        print(f"[{query}] 검색 중 예상치 못한 오류: {e}")
        return []



# 6. 특정 동영상의 댓글 수집 함수
def get_video_comments(youtube, video_id, max_comments_per_video=100):
    """
    주어진 videoId의 댓글을 최대 max_comments_per_video개까지 수집합니다.

    Parameters:
        youtube (Resource): YouTube API 클라이언트
        video_id (str): 대상 YouTube 영상의 ID
        max_comments_per_video (int): 수집할 최대 댓글 수

    Returns:
        List[Dict]: 댓글 정보를 담은 딕셔너리 리스트
    """
    if not youtube:
        print("YouTube 클라이언트가 유효하지 않습니다.")
        return []

    comments = []
    next_page_token = None
    fetched_count = 0

    print(f"[{video_id}] 댓글 수집 시작 (최대 {max_comments_per_video}개)...")

    while fetched_count < max_comments_per_video:
        try:
            max_batch = min(100, max_comments_per_video - fetched_count)

            response = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=max_batch,
                pageToken=next_page_token,
                textFormat="plainText"
            ).execute()

            items = response.get("items", [])
            for item in items:
                snippet = item.get("snippet", {}).get("topLevelComment", {}).get("snippet", {})
                comment_id = item.get("snippet", {}).get("topLevelComment", {}).get("id")
                comment_text = snippet.get("textDisplay")
                author = snippet.get("authorDisplayName", "Unknown")
                published_at = snippet.get("publishedAt")

                if comment_text:
                    comments.append({
                        "video_id": video_id,
                        "comment_id": comment_id,
                        "comment_text": comment_text,
                        "comment_author": author,
                        "comment_published_at": published_at
                    })
                    fetched_count += 1

                if fetched_count >= max_comments_per_video:
                    break

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break

        except googleapiclient.errors.HttpError as e:
            error_msg = str(e)
            if 'commentsDisabled' in error_msg:
                print(f"[{video_id}] 댓글 기능 비활성화")
            elif e.resp.status == 403:
                print(f"[{video_id}] 접근 권한 문제 또는 API 할당량 초과")
            else:
                print(f"[{video_id}] API 오류 발생: {e}")
            break

        except Exception as e:
            print(f"[{video_id}] 예외 발생: {e}")
            break

    print(f"[{video_id}] 댓글 {fetched_count}개 수집 완료.")
    return comments

In [ ]:
# 7. 메인 실행 로직
if __name__ == "__main__":
    if API_KEY: # API 키가 유효할 때만 실행
        youtube_client = get_youtube_client(API_KEY)

        if youtube_client:
            # --- 사용자 입력 ---
            search_query = input("댓글을 수집할 제품명을 입력하세요 (예: CU 밤티라미수): ")
            num_videos_to_search = int(input("검색할 관련 영상의 최대 개수를 입력하세요 (예: 5): "))
            num_comments_per_video = int(input("각 영상에서 수집할 최대 댓글 수를 입력하세요 (예: 100): "))
            # -----------------

            print(f"\n'{search_query}' 관련 영상 검색 및 댓글 수집을 시작합니다...")

            # 1. 관련 영상 검색
            videos_found = search_videos(youtube_client, search_query, num_videos_to_search)

            import time  # 이미 되어있다면 생략

            all_comments_data = []
            start_time = time.time()  # ▶ 수집 시작 시각 기록


            all_comments_data = []

            if videos_found:
                # 2. 각 영상별 댓글 수집
                for video in videos_found:
                    video_id = video['video_id']
                    video_title = video['video_title']
                    print(f"\n영상 제목: {video_title} (ID: {video_id})")

                    comments = get_video_comments(youtube_client, video_id, num_comments_per_video)

                    # 수집된 댓글에 video_id와 video_title 정보 추가
                    for comment in comments:
                        comment['video_id'] = video_id
                        comment['video_title'] = video_title

                    all_comments_data.extend(comments)

                print(f"\n총 {len(all_comments_data)}개의 댓글을 수집했습니다.")
                end_time = time.time()
                elapsed = end_time - start_time
                print(f"⏱️ 댓글 수집 소요 시간: {elapsed:.2f}초")

                # 3. 결과를 Pandas DataFrame으로 변환
                if all_comments_data:
                    df_comments = pd.DataFrame(all_comments_data)

                    # 보기 좋게 컬럼 순서 조정
                    df_comments = df_comments[['video_id', 'video_title', 'comment_id', 'comment_published_at', 'comment_author', 'comment_text']]

                    print("\n--- 수집된 댓글 데이터 샘플 ---")
                    print(df_comments.head())

                    # 4. (선택) 결과를 CSV 파일로 저장
                    save_csv = input("\n수집된 댓글 데이터를 CSV 파일로 저장하시겠습니까? (y/n): ")
                    if save_csv.lower() == 'y':
                        csv_filename = f"{search_query.replace(' ', '_')}_comments.csv"
                        df_comments.to_csv(csv_filename, index=False, encoding='utf-8-sig')
                        print(f"'{csv_filename}'으로 저장되었습니다.")
                else:
                    print("\n수집된 댓글 데이터가 없습니다.")
            else:
                print(f"'{search_query}'에 대한 관련 영상을 찾지 못했습니다.")
        else:
            print("YouTube API 클라이언트 초기화에 실패했습니다.")
    else:
        print("YouTube API 키가 설정되지 않아 스크립트를 실행할 수 없습니다.")